In [1]:
#import libraries

import requests
from bs4 import BeautifulSoup
import glob
from datetime import datetime
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import calmap
%matplotlib inline

In [2]:
# Ministry of Health and Family Welfare url
URL = 'https://www.mohfw.gov.in'
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

html_thead = soup.find_all('thead')[-1]
html_th = [tr for tr in html_thead.find_all('tr')]

html_tbody = soup.find_all('tbody')[-1]
html_text = [tr for tr in html_tbody.find_all('tr')]

headings = []

for tr in html_th:
    th = tr.find_all(['th'])
    row = [i.text for i in th]
    headings.append(row)

print(headings)

content = []
for tr in html_text:
    td = tr.find_all(['td'])
    row = [i.text for i in td]
    content.append(row)
    
#print(content)

[['S. No.', 'Name of State / UT', 'Active Cases*', 'Cured/Discharged/Migrated*', 'Deaths**', 'Total Confirmed cases*']]


In [3]:
data= pd.DataFrame(content[:-6], columns=headings[0])

In [4]:
data

S. No.                        Name of State / UT Active Cases*  \
0       1               Andaman and Nicobar Islands            58   
1       2                            Andhra Pradesh         31763   
2       3                         Arunachal Pradesh           632   
3       4                                     Assam          8675   
4       5                                     Bihar         10506   
5       6                                Chandigarh           254   
6       7                              Chhattisgarh          1709   
7       8  Dadra and Nagar Haveli and Daman and Diu           257   
8       9                                     Delhi         14954   
9      10                                       Goa          1607   
10     11                                   Gujarat         11915   
11     12                                   Haryana          6117   
12     13                          Himachal Pradesh           609   
13     14                         Jammu and Kashmir          6831   
14     15                                 Jharkhand          3397   
15     16                                 Karnataka         47075   
16     17                                    Kerala          8825   
17     18                                    Ladakh           186   
18     19                            Madhya Pradesh          7236   
19     20                               Maharashtra        137282   
20     21                                   Manipur           632   
21     22                                 Meghalaya           433   
22     23                                   Mizoram           138   
23     24                                  Nagaland           598   
24     25                                    Odisha          5977   
25     26                                Puducherry           900   
26     27                                    Punjab          3391   
27     28                                 Rajasthan          8387   
28     29                                    Sikkim           326   
29     30                                Tamil Nadu         51765   
30     31                                 Telangana         11155   
31     32                                   Tripura          1408   
32     33                               Uttarakhand          1894   
33     34                             Uttar Pradesh         20825   

   Cured/Discharged/Migrated* Deaths** Total Confirmed cases*  
0                         163        0                    221  
1                       32127      823                  64713  
2                         314        3                    949  
3                       18033       64                  26772  
4                       19646      217                  30369  
5                         526       13                    793  
6                        4230       29                   5968  
7                         474        2                    733  
8                      107650     3719                 126323  
9                        2541       28                   4176  
10                      37260     2224                  51399  
11                      21697      372                  28186  
12                       1105       11                   1725  
13                       8607      273                  15711  
14                       3024       64                   6485  
15                      27239     1519                  75833  
16                       6162       45                  15032  
17                       1018        2                   1206  
18                      16836      770                  24842  
19                     187769    12556                 337607  
20                       1428        0                   2060  
21                         77        4                    514  
22                        179        0                    317  
23                        486        0             

In [5]:
# drop S.NO column
data = data.drop('S. No.', axis=1)

In [6]:
data.head()

Name of State / UT Active Cases* Cured/Discharged/Migrated*  \
0  Andaman and Nicobar Islands            58                        163   
1               Andhra Pradesh         31763                      32127   
2            Arunachal Pradesh           632                        314   
3                        Assam          8675                      18033   
4                        Bihar         10506                      19646   

  Deaths** Total Confirmed cases*  
0        0                    221  
1      823                  64713  
2        3                    949  
3       64                  26772  
4      217                  30369

In [7]:
#rename column names
data = data.rename(columns={'Name of State / UT':'States'})
data = data.rename(columns=lambda x: re.sub('Total Confirmed cases \(Including .* foreign Nationals\) ',
                                                      'Total Cases',x))
data = data.rename(columns=lambda x: re.sub('Deaths \( more than 70% cases due to comorbidities \)',
                                                      'Deaths', x))
data = data.rename(columns={'Total Confirmed cases*': 'Total Cases'})
data = data.rename(columns={'Deaths**':'Death'})
data = data.rename(columns={'Cured/Discharged/Migrated*':'Recovered'})
data = data.rename(columns={'Deaths':'Death'})
data = data.rename(columns={'Active Cases*':'Active Cases'})

# remove extra characters from 'Name of State/UT' column
data['States'] = data['States'].str.replace('#', '')

# remove extra characters from 'Name of State/UT' column
data['Death'] = data['Death'].str.replace('#', '')

# drop active cases colum
data = data.drop('Active Cases', axis=1)

In [8]:
data.head()

States Recovered Death Total Cases
0  Andaman and Nicobar Islands       163     0         221
1               Andhra Pradesh     32127   823       64713
2            Arunachal Pradesh       314     3         949
3                        Assam     18033    64       26772
4                        Bihar     19646   217       30369

In [9]:
#add todays date and create new column
now  = datetime.now()
data['Date'] = now.strftime("%m/%d/%Y") 
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data.head()

States Recovered Death Total Cases       Date
0  Andaman and Nicobar Islands       163     0         221 2020-07-23
1               Andhra Pradesh     32127   823       64713 2020-07-23
2            Arunachal Pradesh       314     3         949 2020-07-23
3                        Assam     18033    64       26772 2020-07-23
4                        Bihar     19646   217       30369 2020-07-23

In [10]:
#add latitude and longitude
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 
       'Maharashtra':19.7515, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670}

long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 
        'Maharashtra':75.7139, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662}

data['Latitude'] = data['States'].map(lat)
data['Longitude'] = data['States'].map(long)

In [11]:
data.head()

States Recovered Death Total Cases       Date  \
0  Andaman and Nicobar Islands       163     0         221 2020-07-23   
1               Andhra Pradesh     32127   823       64713 2020-07-23   
2            Arunachal Pradesh       314     3         949 2020-07-23   
3                        Assam     18033    64       26772 2020-07-23   
4                        Bihar     19646   217       30369 2020-07-23   

   Latitude  Longitude  
0   11.7401    92.6586  
1   15.9129    79.7400  
2   28.2180    94.7278  
3   26.2006    92.9376  
4   25.0961    85.3131

In [12]:
my_folder = './.daily_update/'
if not os.path.exists(my_folder):
    os.makedirs(my_folder)

file_name = my_folder + now.strftime("%Y_%m_%d") + '.csv'

In [13]:
data.to_csv(file_name, index=False)

In [14]:
csv_files = glob.glob(my_folder + '*.csv')

all_data = []
for i in csv_files:
    temp = pd.read_csv(i)
    all_data.append(temp)
    

In [15]:
final_data = pd.concat(all_data, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)
final_data['Date'] = pd.to_datetime(final_data['Date'])
#final_data.drop(final_data.index[[1005]])
final_data['Active'] = final_data['Total Cases'] - final_data['Recovered'] - final_data['Death']
final_data = final_data.sort_values(['Date', 'States']).reset_index(drop=True)
final_data

States  Recovered  Death  Total Cases       Date  \
0     Andaman and Nicobar Islands         11      0           16 2020-04-21   
1                  Andhra Pradesh         96     22          757 2020-04-21   
2               Arunachal Pradesh          1      0            1 2020-04-21   
3                           Assam         19      1           35 2020-04-21   
4                           Bihar         42      2          114 2020-04-21   
...                           ...        ...    ...          ...        ...   
2664                   Tamil Nadu     131583   3144       186492 2020-07-23   
2665                    Telangana      37666    438        49259 2020-07-23   
2666                      Tripura       2032      9         3449 2020-07-23   
2667                Uttar Pradesh      33500   1263        55588 2020-07-23   
2668                  Uttarakhand       3349     57         5300 2020-07-23   

      Latitude  Longitude  Active  
0      11.7401    92.6586       5  
1      15.9129    79.7400     639  
2      28.2180    94.7278       0  
3      26.2006    92.9376      15  
4      25.0961    85.3131      70  
...        ...        ...     ...  
2664   11.1271    78.6569   51765  
2665       NaN        NaN   11155  
2666   23.9408    91.9882    1408  
2667   26.8467    80.9462   20825  
2668   30.0668    79.0193    1894  

[2669 rows x 8 columns]

In [16]:
final_data.to_csv('COVID19INDIA.csv', index=False)